In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer ,CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB , MultinomialNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,hamming_loss
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN

In [1]:
import pandas as pd
import gzip

with gzip.open('cleaned_text.csv', 'rt', encoding='utf-8') as file:
    df = pd.read_csv(file)
df = df[['cleaned_text' , 'id' , 'label']]

In [2]:
df.head()

,cleaned_text,id,label
0,keep glove hat coat jacket togeth need entrywa...,122885,Assembly Required
1,keep glove hat coat jacket togeth need entrywa...,122885,Finish
2,keep glove hat coat jacket togeth need entrywa...,122885,Hardware Included
3,home dynamix serendip ivori ft x ft area r...,188958,Commercial / Residential
4,home dynamix serendip ivori ft x ft area r...,188958,Features


In [4]:

# Perform one-hot encoding on the 'label' column
df_encoded = pd.get_dummies(df, columns=['label'] , prefix = '')

# Group by 'text' and aggregate using max to get 1s and 0s
df_grouped = df_encoded.groupby('id').max().reset_index()
df_grouped.columns = df_grouped.columns.str.replace(r'^.*_', '')
# Print the resulting DataFrame
df_grouped.sample(1)

,id,text,Assembly Required,Color,Commercial / Residential,ENERGY STAR Certified,Features,Finish,Flooring Product Type,Hardware Included,Included,Indoor/Outdoor,Package Quantity,Shape,Tools Product Type,Voltage (volts),Wattage (watts)
13164,153076,exclus intric detail planter perfect center pi...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [8]:
df_grouped[df['id']==122885]

,id,text,Assembly Required,Color,Commercial / Residential,ENERGY STAR Certified,Features,Finish,Flooring Product Type,Hardware Included,Included,Indoor/Outdoor,Package Quantity,Shape,Tools Product Type,Voltage (volts),Wattage (watts)
0,100003,classic architectur meet contemporari design e...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,100004,grape solar watt polycrystallin pv solar pane...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
2,100006,achiev delici result almost effortless whirlpo...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [4]:
tfidf = TfidfVectorizer(max_features=5000 ,ngram_range=(1,4) )
Xfeatures = tfidf.fit_transform(df_grouped['text']).toarray()

In [5]:
y = df_grouped[['Assembly Required', 'Color', 'Commercial / Residential',
       'ENERGY STAR Certified', 'Features', 'Finish', 'Flooring Product Type',
       'Hardware Included', 'Included', 'Indoor/Outdoor', 'Package Quantity',
       'Shape', 'Tools Product Type', 'Voltage (volts)', 'Wattage (watts)']]

In [6]:
# Datasplit
X_train,X_test,y_train,y_test = train_test_split(Xfeatures,y,test_size=0.3,random_state=42)

In [7]:
def build_model(model_name , model ,xtrain,ytrain,xtest,ytest,mlb_estimator = LabelPowerset):
    output= []
    
    output.append(model_name)
    clf = mlb_estimator(model)
    clf.fit(xtrain,ytrain)
    clf_predictions = clf.predict(xtest)
    train_acc = clf.score(X_train ,y_train)
    output.append(train_acc)

    test_acc = accuracy_score(ytest,clf_predictions)
    output.append(test_acc)
    
    ham = hamming_loss(ytest,clf_predictions) * 100
    output.append(ham)
    print(f'Train : {train_acc} || Test : {test_acc} || Hamming_score : {ham}')
    #print(model)
    return output 

In [8]:
model_dict = {
    'logistic_reg': LogisticRegression()        
}
    
model_output = []
for model_name,model in model_dict.items():
    model_output.append(build_model(model_name, model,X_train,y_train,X_test,y_test))

Train : 0.8574816138566225 || Test : 0.8003367813326918 || Hamming_score : 2.5739716141448157
